# GP2 CNVs Pipeline

In [2]:
import pandas as pd
import os
import shutil
import numpy as np
import glob
from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm

# from QC.utils import shell_do

In [6]:
# Supress copy warning.
pd.options.mode.chained_assignment = None

In [7]:
def shell_do(command, log=False, return_log=False):
    print(f'Executing: {(" ").join(command.split())}', file=sys.stderr)

    res=subprocess.run(command.split(), stdout=subprocess.PIPE)

    if log:
        print(res.stdout.decode('utf-8'))
    if return_log:
        return(res.stdout.decode('utf-8'))

In [8]:
wd = '/YOUR/DIRECTORY'

### Begin Pipeline

In [13]:
############ SET THIS EVERY RELEASE ############
release = '5'

key_file = f'/data/GP2/clinical/master_key/GP2_master_key_FINAL_release5_n26728.txt'
key = pd.read_csv(f'{key_file}')
key.loc[:,'IID'] = key.loc[:,'SentrixBarcode_A'].astype(str) + '_' + key.loc[:,'SentrixPosition_A'].astype(str)
key.loc[:,'FID'] = "0"

swarm_scripts_dir = f'{wd}/swarm'
samples_list = key.loc[:,'filename'].unique()
barcodes_list = list(set([x.split('_')[0] for x in samples_list]))

raw_geno_path = '/data/GP2/raw_genotypes'
snp_metrics_path = f'{raw_geno_path}/snp_metrics'
cnv_path = f'{wd}/original_CNVs' # personal version
os.makedirs(cnv_path, exist_ok=True)

# cnv_path = f'/data/GP2/raw_genotypes/cnvs/release{release}' # new release version
# covar_path = f'{cnv_path}/key/release{release}_covar.csv'

ancestry_labels = [x.split('/')[-1].replace('.bed','').split('_')[-1] for x in glob.glob(f'/data/GP2/quality_control/release{release}/genotype_qc/GP2_round5_APRIL_2023_*.bed') if '_maf_hwe' not in x]

## already have these for previous releases - uncomment for new release
# key[['FID','GP2sampleID']].to_csv(f'{cnv_path}/key/release{release}.samples', sep='\t', header=False, index=False)
# key[['GP2sampleID','IID']].to_csv(f'{cnv_path}/key/release{release}_sample_id_key.csv')
# release_covars = key.loc[:,['FID', 'GP2sampleID','sex_for_qc', 'age', 'age_of_onset']]
# release_covars.to_csv(covar_path)

cnv_types = ['PERCENT_BAF_INSERTION','PERCENT_L2R_DELETION','PERCENT_L2R_DUPLICATION']

In [10]:
!ls {snp_metrics_path}/{barcodes_list[0]}/snp_metrics_{barcodes_list[0]}

'Sample_ID=204835450068_R01C01'  'Sample_ID=204835450068_R05C01'
'Sample_ID=204835450068_R02C01'  'Sample_ID=204835450068_R06C01'
'Sample_ID=204835450068_R03C01'  'Sample_ID=204835450068_R07C01'
'Sample_ID=204835450068_R04C01'  'Sample_ID=204835450068_R08C01'


In [26]:
# Check if all samples in the list have a parquet created with SNP metrics
exist_dir = []
missing_dir = []

for sample in samples_list:
    code = sample.split('_')[0]
    metrics_file = f'{snp_metrics_path}/{code}/snp_metrics_{code}/Sample_ID={sample}'
    
    if os.path.isdir(metrics_file):
        exist_dir.append(sample)
    else:
        missing_dir.append(sample)
        
print(f'metrics exist: {len(exist_dir)}')
print(f'missing metrics: {len(missing_dir)}')

metrics exist: 26578
missing metrics: 150


In [41]:
# check ancestries
total_sample_n = 0
for label in ancestry_labels:

    # will need to change for each release
    gene_name = f'GP2_round5_APRIL_2023_{label}'
    gene_path = f'/data/GP2/quality_control/release{release}/genotype_qc/{gene_name}'
    
    fam = pd.read_csv(f'{gene_path}.fam', sep='\s+', header=None, names=['fid','iid','pat','mat','sex','pheno'])
    pheno_counts = fam.pheno.value_counts()
    ncases = pheno_counts[2]
    ncontrols = pheno_counts[1]
    ntotal = ncases + ncontrols
    total_sample_n += ntotal
    nsnps = !cat {out_path}.bim | wc -l
    print(f'{label} | total_n: {ntotal} | n_cases: {ncases} | n_controls: {ncontrols} | n_snps: {int(nsnps[0])}')
    print()
    
print(f'total samples across ancestries: {total_sample_n}')

AAC | total_n: 1320 | n_cases: 289 | n_controls: 1031 | n_snps: 984555

AFR | total_n: 2343 | n_cases: 855 | n_controls: 1488 | n_snps: 984555

AJ | total_n: 1129 | n_cases: 742 | n_controls: 387 | n_snps: 984555

AMR | total_n: 550 | n_cases: 345 | n_controls: 205 | n_snps: 984555

CAS | total_n: 485 | n_cases: 202 | n_controls: 283 | n_snps: 984555

EAS | total_n: 3567 | n_cases: 1223 | n_controls: 2344 | n_snps: 984555

EUR | total_n: 15359 | n_cases: 9891 | n_controls: 5468 | n_snps: 984555

FIN | total_n: 13 | n_cases: 10 | n_controls: 3 | n_snps: 984555

MDE | total_n: 139 | n_cases: 113 | n_controls: 26 | n_snps: 984555

SAS | total_n: 240 | n_cases: 104 | n_controls: 136 | n_snps: 984555

total samples across ancestries: 25145


In [46]:
# skip extra QC, only use QC that checked for duplicates and mild variant pruning
# if use subset of samples, only include the ancestry_labels applicable to those samples

missing_metrics = []
included_samples = []

with open(f'{swarm_scripts_dir}/cnvs.swarm', 'w') as f:
    for label in ancestry_labels:
        
        # will need to change for each release
        geno_name = f'GP2_round5_APRIL_2023_{label}'
        geno_path = f'/data/GP2/quality_control/release{release}/genotype_qc/{geno_name}'
        
        bim_path = f'{geno_path}.bim'
        fam = pd.read_csv(f'{geno_path}.fam', sep='\s+', header=None, names=['FID','IID','pat','mat','sex','pheno'])
        fam_key = fam.merge(key, left_on='IID', right_on='GP2sampleID', how='left')
        
        # may want to change naming convention of directory where CNVs will be output: mine is cnv_calls_no_qc
        label_dir = f'{cnv_path}/cnv_calls_no_qc/{label}'
        os.makedirs(label_dir, exist_ok=True)

        for sample in fam_key.filename.unique():
            if pd.notnull(sample): # catch some sample IDs that are NaN
                code = sample.split('_')[0]
                mfile = f'{snp_metrics_path}/{code}/snp_metrics_{code}/Sample_ID={sample}'
                
                if os.path.isdir(mfile):
                    cnv_out = f'{label_dir}/CNV_{label}_{sample}.parquet'
                    intervals = '/data/GP2/utils/ref_dir/glist_hg38_intervals.csv'
            
                    cmd = f'\
                            python run_cnv_pipeline.py \
                            --metrics {mfile} \
                            --bim {bim_path} \
                            --out_path {cnv_out} \
                            --intervals {intervals} \
                            --min_variants 10 \
                            --kb_window 250 \
                            --min_gentrain 0.2'
            
                    f.write(f'{cmd}\n')
                    included_samples.append(sample)
            
                else:
                    missing_metrics.append(sample)
        
f.close()

In [47]:
# make sure before actually running swarm script
print(len(included_samples))

24935


In [34]:
!swarm -f {swarm_scripts_dir}/cnvs.swarm -g 16 -t 8 --time=00:15:00 --logdir {swarm_scripts_dir}/logs --module python/3.9 --gres=lscratch:20 --partition=norm

13171067


In [ ]:
# write dosages per chromosome per ancestry per cnv-type (1 .csv per ancestry per cnv-type)

with open(f'{swarm_scripts_dir}/cnv_dosages.swarm', 'w') as f:
    for label in ancestry_labels:
        label_dir = f'{cnv_path}/dosages/{label}'
        os.makedirs(label_dir, exist_ok=True)

        # if you want to change naming convention of cnv_calls_no_qc
        cnv_files_list = glob.glob(f'{cnv_path}/cnv_calls_no_qc/{label}/CNV_{label}_*.parquet')
        cnv_files_df = pd.DataFrame({'filename': cnv_files_list})
        cnv_files_df.to_csv(f'{label_dir}/CNV_{label}_files.csv', header=False, index=False)

        for cnv_type in cnv_types:

            cmd = f'\
                    python run_cnv_dosage_pipeline.py \
                    --files {label_dir}/CNV_{label}_files.csv \
                    --label {label} \
                    --cnv_type {cnv_type} \
                    --out_path {label_dir}/CNV_{label}_{cnv_type}.csv'
            f.write(f'{cmd}\n')
f.close()

In [ ]:
!swarm -f {swarm_scripts_dir}/cnv_dosages.swarm -g 8 -t 4 --logdir {swarm_scripts_dir}/logs --module python/3.9 --gres=lscratch:20 --partition=norm

### If want to double-check CNVs for 1 sample

In [35]:
# check 1 European sample
# may need to change directory to your naming conventions
test_cnv = pd.read_parquet(f'{wd}/original_CNVs/cnv_calls_qc/EUR/CNV_EUR_{samples_list[0]}.parquet')

# not CNVs
display(test_cnv[test_cnv.PERCENT_BAF_INSERTION.isnull()])

# CNVs
display(test_cnv[test_cnv.NUM_VARIANTS>10])

,INTERVAL,NUM_VARIANTS,PERCENT_BAF_INSERTION,PERCENT_L2R_DELETION,PERCENT_L2R_DUPLICATION,START_PLUS_WINDOW,START,STOP,STOP_PLUS_WINDOW
0,A1BG,2,NaN,NaN,NaN,58096805,58346805,58353499,58603499
1,A1BG-AS1,2,NaN,NaN,NaN,58101969,58351969,58355183,58605183
2,A1CF,1,NaN,NaN,NaN,50549408,50799408,50885675,51135675
3,A2M,2,NaN,NaN,NaN,8817707,9067707,9115962,9365962
4,A2M-AS1,2,NaN,NaN,NaN,8815176,9065176,9068055,9318055
...,...,...,...,...,...,...,...,...,...
25618,ZYG11A,2,NaN,NaN,NaN,52592510,52842510,52894575,53144575
25619,ZYG11B,2,NaN,NaN,NaN,52476458,52726458,52827341,53077341
25620,ZYX,4,NaN,NaN,NaN,143131266,143381266,143391111,143641111
25621,ZZEF1,9,NaN,NaN,NaN,3754444,4004444,4142959,4392959


,INTERVAL,NUM_VARIANTS,PERCENT_BAF_INSERTION,PERCENT_L2R_DELETION,PERCENT_L2R_DUPLICATION,START_PLUS_WINDOW,START,STOP,STOP_PLUS_WINDOW
85,ABCG1,13,0.0,0.000000,0.000000,41949688,42199688,42297244,42547244
121,ABR,12,0.0,0.000000,0.000000,753517,1003517,1187322,1437322
190,ACPT,16,0.0,0.062500,0.187500,50540414,50790414,50795224,51045224
237,ACTR3B,13,0.0,0.000000,0.230769,152509748,152759748,152855379,153105379
291,ADAMTS2,13,0.0,0.000000,0.076923,178860850,179110850,179345430,179595430
...,...,...,...,...,...,...,...,...,...
25314,ZNF556,13,0.0,0.000000,0.076923,2617334,2867334,2878503,3128503
25476,ZNF761,17,0.0,0.058824,0.117647,53181973,53431973,53458261,53708261
25479,ZNF765,16,0.0,0.062500,0.125000,53145143,53395143,53412009,53662009
25515,ZNF813,17,0.0,0.058824,0.117647,53217734,53467734,53494292,53744292


In [25]:
# double check counts in the ancestry labels

for label in ancestry_labels:
# for label in ['EUR']: # if want specific label
    cnv_dir_ = f'{cnv_path}/cnv_calls/{label}/'
    label_cnvs_list = glob.glob(f'{cnv_dir_}/*.parquet')